In [1]:
import pandas as pd
import requests

In [7]:
response = requests.get('https://www.balldontlie.io/api/v1/players?per_page=100&page=1')

In [8]:
response.text

'{"data":[{"id":149,"first_name":"Keenan","height_feet":null,"height_inches":null,"last_name":"Evans","position":"G","team":{"id":9,"abbreviation":"DET","city":"Detroit","conference":"East","division":"Central","full_name":"Detroit Pistons","name":"Pistons"},"weight_pounds":null},{"id":382,"first_name":"Billy","height_feet":null,"height_inches":null,"last_name":"Preston","position":"F","team":{"id":6,"abbreviation":"CLE","city":"Cleveland","conference":"East","division":"Central","full_name":"Cleveland Cavaliers","name":"Cavaliers"},"weight_pounds":null},{"id":448,"first_name":"Gary","height_feet":null,"height_inches":null,"last_name":"Trent Jr.","position":"G","team":{"id":25,"abbreviation":"POR","city":"Portland","conference":"West","division":"Northwest","full_name":"Portland Trail Blazers","name":"Trail Blazers"},"weight_pounds":null},{"id":46391370,"first_name":"Jake","height_feet":null,"height_inches":null,"last_name":"Carter","position":"","team":{"id":50,"abbreviation":"DN","ci

In [10]:
(pd.DataFrame(response.json()['data']))[["id", "first_name", "last_name", "position", "height_feet", "height_inches", "weight_pounds"]]

,id,first_name,last_name,position,height_feet,height_inches,weight_pounds
0,149,Keenan,Evans,G,NaN,NaN,NaN
1,382,Billy,Preston,F,NaN,NaN,NaN
2,448,Gary,Trent Jr.,G,NaN,NaN,NaN
3,46391370,Jake,Carter,,NaN,NaN,NaN
4,2144,Rakeem,Christmas,,NaN,NaN,NaN
...,...,...,...,...,...,...,...
95,46389602,Tommy,Byrnes,,NaN,NaN,NaN
96,83,Clint,Capela,C,6.0,10.0,240.0
97,2948,Dell,Curry,,NaN,NaN,NaN
98,666541,Luguentz,Dort,G,NaN,NaN,NaN


In [11]:
# Creating a DataFrame
df = pd.DataFrame()

In [12]:
if response.status_code == 200:
    for i in range(1, 54):
        print(i)
        response = requests.get('https://www.balldontlie.io/api/v1/players?per_page=100&page={}'.format(i))
        temp_df_1 = (pd.DataFrame(response.json()['data']))[["id", "first_name", "last_name", "position", "height_feet", "height_inches", "weight_pounds"]]
        temp_df_2 = pd.DataFrame(list(pd.DataFrame(response.json()['data'])['team']))
        temp_df_2.rename(columns ={'id':'team_id', 'abbreviation':'team_abbreviation', 'city':'team_city','conference':'team_conference','division':'team_division','full_name':'team_full_name','name':'team_name'}, inplace = True)
        temp_df_3 = pd.concat([temp_df_1, temp_df_2], axis = 1)
        df = df.append(temp_df_3, ignore_index=True)
else:
    print('Error', response.status_code)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54


KeyError: "None of [Index(['id', 'first_name', 'last_name', 'position', 'height_feet',\n       'height_inches', 'weight_pounds'],\n      dtype='object')] are in the [columns]"

In [14]:
df.to_csv("Player_info.csv")

In [15]:
df

,id,first_name,last_name,position,height_feet,height_inches,weight_pounds,team_id,team_abbreviation,team_city,team_conference,team_division,team_full_name,team_name
0,149,Keenan,Evans,G,NaN,NaN,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons
1,382,Billy,Preston,F,NaN,NaN,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
2,448,Gary,Trent Jr.,G,NaN,NaN,NaN,25,POR,Portland,West,Northwest,Portland Trail Blazers,Trail Blazers
3,46391370,Jake,Carter,,NaN,NaN,NaN,50,DN,,,,Denver Nuggets,Denver Nuggets
4,2144,Rakeem,Christmas,,NaN,NaN,NaN,12,IND,Indiana,East,Central,Indiana Pacers,Pacers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5224,492,Ante,Zizic,C,6.0,11.0,254.0,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers
5225,2639,Jim,Zoet,,NaN,NaN,NaN,9,DET,Detroit,East,Central,Detroit Pistons,Pistons
5226,46405957,Bill,Zopf,,NaN,NaN,NaN,17,MIL,Milwaukee,East,Central,Milwaukee Bucks,Bucks
5227,493,Ivica,Zubac,C,7.0,1.0,240.0,13,LAC,LA,West,Pacific,LA Clippers,Clippers
